In [8]:
import pandas as pd
import numpy as np
# This file contains all required utilities to build resolvings sets and probabilistic methods
from Utils.resolving import *
from Utils.dataframes import Dict_to_df
from Utils.SVM  import SVC_model
from Utils.plots import averaging_plot

SENTI_DATA_PATH= "../Files/Full_Vader_Senti.txt"

In [9]:
def generate_noise_sentences(n = 350, lam = 50, noise_fraction = 0.01):
    imported_vader_words = []
    negative_vader_list = []
    neutral_vader_list = []
    positive_vader_list = []


    #reading in the words. Change the filepath to suit where yours is stored
    x = open(SENTI_DATA_PATH, "r")
    #reading in the words. Change the filepath to suit where yours is stored
    for line in x:
        
        l = line.strip('\n')
        l = l.split("\t")
        imported_vader_words.append(l)
        
    x.close()

    for y in imported_vader_words : #sorting the words by tone
        if (-1<= float(y[1]) <= 1):
            neutral_vader_list.append(y[0]) 
        elif (float(y[1]) < -1):
            negative_vader_list.append(y[0])
        else :
            positive_vader_list.append(y[0])


    shorten_pos = random.sample(positive_vader_list, n) #gets random n words w/out duplicates
    shorten_neg = random.sample(negative_vader_list, n) #gets random n words w/out duplicates
    shorten_neutral = random.sample(neutral_vader_list, n) #gets random n words w/out duplicates


    negative_sent_list = []
    for i in range(0,500):
        h = np.random.poisson(lam=lam - lam*noise_fraction, size=None)
        k = np.random.poisson(lam= lam*noise_fraction, size=None)
        negative_sent_list.append(random.sample(shorten_neg, h) +random.sample(shorten_neutral, k))

    positive_sent_list = []
    for i in range(0,500):
        #changes so poisson is still same mean with addition
        h = np.random.poisson(lam=lam - lam*noise_fraction, size=None)
        k = np.random.poisson(lam= lam*noise_fraction, size=None)
        positive_sent_list.append(random.sample(shorten_pos, h)+random.sample(shorten_neutral, k))


    final_list = negative_sent_list + positive_sent_list

    #getting wordlist 
    wordlist_noise = {x for l in final_list for x in l}
    wordlist_noise_prime = {x for l in final_list for x in l}

    #generating resolving set. note that the "prime" is for the compliment resolving set, and can be deleted if you don't want to deal with that
    resolving_noise = Random_Resolving_Set(wordlist_noise)
    resolving_noise_prime = Random_Resolving_Set_Compliment(wordlist_noise_prime)

    #generating the Jaccard Matrices
    JacMatrix_noise = JVecEMatrix(resolving_noise,final_list)
    JacMatrix_noise_prime = JVecEMatrix(resolving_noise_prime,final_list)

    labeled_dic_noise = {}
    labeled_dic_noise_prime = {}

    #this just stores all the vectors of the Jmatrix in a dictionary with key vec, and tone corresponding to its tone. 
    for i in range(0,len(JacMatrix_noise)):
        if (i<500):
            labeled_dic_noise[i] = {'vec': JacMatrix_noise[i], 'tone': "negative"}
        elif (500<i<1000):
            labeled_dic_noise[i] = {'vec': JacMatrix_noise[i], 'tone': "positive"}
        else:
            labeled_dic_noise[i] = {'vec': JacMatrix_noise[i], 'tone': "neutral"} #for now this never triggers, but could for tri tone cross contam


    for i in range(0,len(JacMatrix_noise_prime)):
        if (i<500):
            labeled_dic_noise_prime[i] = {'vec': JacMatrix_noise_prime[i], 'tone': "negative"}
        elif (500<i<1000):
            labeled_dic_noise_prime[i] = {'vec': JacMatrix_noise_prime[i], 'tone': "positive"}
        else:
            labeled_dic_noise_prime[i] = {'vec': JacMatrix_noise_prime[i], 'tone': "neutral"}#for now this never triggers, but could for tri tone cross contam

    return Dict_to_df(labeled_dic_noise)

In [5]:
noise_df, tones = generate_noise_sentences()

308


## Noisey Sentences Averaging Plot